Функции для обучения моделей, запускавшиеся в Yandex DataSphere. Передаваемые и возвращаемые переменные в соответсвии с заданием

## Практика 1. Обучение многослойной сети

In [3]:
from keras.datasets import fashion_mnist
from keras.layers import Dense
from keras.models import Sequential
import numpy as np

def load_train(path):
    features_train = np.load(path + 'train_features.npy')
    target_train = np.load(path + 'train_target.npy')
    features_train = features_train.reshape(
        features_train.shape[0], 28 * 28) / 255.
    return features_train, target_train


def create_model(input_shape):
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=input_shape))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    return model


def train_model(model, train_data, test_data, batch_size=32, epochs=20,
                steps_per_epoch=None, validation_steps=None):

    features_train, target_train = train_data
    features_test, target_test = test_data
    model.fit(features_train, target_train,
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    return model

In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28 * 28) / 255
x_test = x_test.reshape(x_test.shape[0], 28 * 28) / 255

print(x_train.shape, y_train.shape)
print(x_train.min(), x_train.max())
print(x_test.shape, y_test.shape)
print(x_test.min(), x_test.max())
print(np.unique(y_train))

(60000, 784) (60000,)
0.0 1.0
(10000, 784) (10000,)
0.0 1.0
[0 1 2 3 4 5 6 7 8 9]


In [5]:
model = create_model((784,))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [6]:
train_model(model, (x_train, y_train), (x_test, y_test))

Epoch 1/20
1875/1875 - 4s - loss: 0.6860 - acc: 0.7730 - val_loss: 0.5148 - val_acc: 0.8235 - 4s/epoch - 2ms/step
Epoch 2/20
1875/1875 - 3s - loss: 0.4751 - acc: 0.8330 - val_loss: 0.4803 - val_acc: 0.8269 - 3s/epoch - 2ms/step
Epoch 3/20
1875/1875 - 3s - loss: 0.4305 - acc: 0.8502 - val_loss: 0.4423 - val_acc: 0.8405 - 3s/epoch - 2ms/step
Epoch 4/20
1875/1875 - 3s - loss: 0.4044 - acc: 0.8581 - val_loss: 0.4358 - val_acc: 0.8435 - 3s/epoch - 2ms/step
Epoch 5/20
1875/1875 - 3s - loss: 0.3841 - acc: 0.8646 - val_loss: 0.4343 - val_acc: 0.8489 - 3s/epoch - 2ms/step
Epoch 6/20
1875/1875 - 3s - loss: 0.3680 - acc: 0.8707 - val_loss: 0.4078 - val_acc: 0.8552 - 3s/epoch - 2ms/step
Epoch 7/20
1875/1875 - 3s - loss: 0.3549 - acc: 0.8750 - val_loss: 0.3979 - val_acc: 0.8573 - 3s/epoch - 2ms/step
Epoch 8/20
1875/1875 - 3s - loss: 0.3429 - acc: 0.8774 - val_loss: 0.3776 - val_acc: 0.8645 - 3s/epoch - 2ms/step
Epoch 9/20
1875/1875 - 3s - loss: 0.3324 - acc: 0.8825 - val_loss: 0.3716 - val_acc: 0.8

In [6]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

## Практика 2. Алгоритм Adam

In [1]:
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)
from keras.optimizers import Adam
import numpy as np


def load_train(path):
    features_train = np.load(path + 'train_features.npy')
    target_train = np.load(path + 'train_target.npy')
    features_train = features_train.reshape(-1, 28, 28, 1) / 255.
    return features_train, target_train


def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(20, (3, 3), strides=1, padding="same", activation="relu",
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Conv2D(10, (3, 3), strides=1, padding="same", activation="relu"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Flatten())
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.005), loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    return model


def train_model(model, train_data, test_data, batch_size=32, epochs=10,
                steps_per_epoch=None, validation_steps=None):

    features_train, target_train = train_data
    features_test, target_test = test_data
    model.fit(features_train, target_train,
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=1, shuffle=True)
    return model

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255
x_test = x_test.reshape(-1, 28, 28, 1) / 255

In [3]:
model = create_model((28, 28, 1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 20)        200       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 20)       80        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 10)        1810      
                                                                 
 dropout (Dropout)           (None, 14, 14, 10)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 10)       4

In [4]:
train_model(model, (x_train, y_train), (x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 10s 4ms/step - loss: 0.4604 - acc: 0.8355 - val_loss: 0.3365 - val_acc: 0.8795
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3292 - acc: 0.8784 - val_loss: 0.3182 - val_acc: 0.8834
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3028 - acc: 0.8879 - val_loss: 0.2923 - val_acc: 0.8917
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2860 - acc: 0.8943 - val_loss: 0.2931 - val_acc: 0.8930
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2731 - acc: 0.8996 - val_loss: 0.2881 - val_acc: 0.8980
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2651 - acc: 0.9027 - val_loss: 0.2754 - val_acc: 0.9035
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2589 - acc: 0.9058 - val_loss: 0.3058 - val_acc: 0.8868
Epoch 8/10
1875/1875 [==============================] - 8s 4m

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Практика 3. Свёрточные сети для классификации фруктов

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.optimizers import Adam


def load_train(path):
    datagen = ImageDataGenerator(
        # validation_split=0.25,
        rescale=1/255.,
        # vertical_flip=True,
        # horizontal_flip=True,
        # rotation_range=90,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # zoom_range=0.1,
    )

    return datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        #subset='training',
    )


def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    model.add(Flatten())
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):
    # raise ValueError(type(train_data), type(test_data))
    model.fit(train_data,
        validation_data=test_data,
        epochs=epochs, batch_size=batch_size,
        steps_per_epoch=steps_per_epoch, #train_data.samples/train_data.batch_size,
        validation_steps=validation_steps, #test_data.samples/test_data.batch_size,
        verbose=2, shuffle=True)
    return model


## Практика 4. ResNet в Keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow import keras

def load_train(path):
    datagen = ImageDataGenerator(
        rescale=1/255.,
        vertical_flip=True,
        horizontal_flip=True,
        rotation_range=90,
        width_shift_range=0.15,
        height_shift_range=0.15,
        zoom_range=0.1,
    )

    return datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
    )


def create_model(input_shape):
    backbone = ResNet50(
        weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
        input_shape=input_shape, 
        include_top=False)
    #backbone.trainable = False
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(12, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=5,
                steps_per_epoch=None, validation_steps=None):
    #raise ValueError(type(train_data), type(test_data))
    model.fit(train_data,
        validation_data=test_data,
        epochs=epochs, batch_size=batch_size,
        steps_per_epoch=steps_per_epoch, #train_data.samples/train_data.batch_size,
        validation_steps=validation_steps, #test_data.samples/test_data.batch_size,
        verbose=2, shuffle=True)
    return model


## Проект

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow import keras
import pandas as pd 


def load_train(path):
    """ Loading of train dataset from 'final_files/' subfolder in `path`
    """
    labels = pd.read_csv(path + 'labels.csv')

    datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1/255.,
        horizontal_flip=True,
        width_shift_range=0.15,
        height_shift_range=0.15,
        zoom_range=0.1,
    )

    return datagen.flow_from_dataframe(
        dataframe = labels,
        directory = path+'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training',
        seed=123,
    )


def load_test(path):
    """ Loading of test dataset from 'final_files/' subfolder in `path`
    """
    labels = pd.read_csv(path + 'labels.csv')

    datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1/255.)

    return datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path+'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',
        seed=123,
    )


def create_model(input_shape):
    """ Creating of ResNet50 model for objects with shape = `input_shape`.  
        Custom fully-connected layer at the top of the network with `relu` activation for predict age value
    """
    backbone = ResNet50(
        weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
        input_shape=input_shape, 
        include_top=False)
    
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))
    model.compile(optimizer=Adam(learning_rate=0.0005), 
                  loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):
    """ Training of `model`
    
    ----
    `train_data` : DataFrameIterator for training dataset
    `test_data` : DataFrameIterator for validation dataset
    `epochs`, `batch_size` : params of keras.Model.fit()
    """
    model.fit(train_data,
        validation_data=test_data,
        epochs=epochs, batch_size=batch_size,
        steps_per_epoch=steps_per_epoch, #train_data.samples/train_data.batch_size,
        validation_steps=validation_steps, #test_data.samples/test_data.batch_size,
        verbose=2, shuffle=True)
    return model